In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import scipy.sparse as scs
import torch
import torch.nn.functional as F

from recsys.utils import col
from recsys.metrics import ndcg_score, get_ndcg, get_apak
from recsys.models.nn import GMF, GMFPointwise

In [ ]:
device = "cuda"
torch.set_default_device(device)

### Last one

In [ ]:
X = pd.read_parquet("../data/ml-1m/split/X_last_one.parquet")
y = pd.read_parquet("../data/ml-1m/split/y_last_one.parquet")

In [ ]:
y_true = y[[col.movie_code]].to_numpy()

test_array = np.hstack([y_true, np.array(y[col.negative].apply(list).tolist())])

test_data = torch.utils.data.DataLoader(
    np.vstack(
        [np.arange(y_true.shape[0]).repeat(test_array.shape[1]), test_array.flatten()]
    ).T,
    batch_size=2048,
    generator=torch.Generator(device=device),
)

In [ ]:
user_movie_matrix = scs.csr_matrix(
    (X[col.rating], (X[col.user_code], X[col.movie_code]))
)

user_movie_matrix[user_movie_matrix.nonzero()] = 1

In [ ]:
negative_samples = user_movie_matrix.sum(axis=0).repeat(
    user_movie_matrix.shape[0], axis=0
)
negative_samples[user_movie_matrix.nonzero()] = -1
negative_samples = np.asarray(np.argsort(negative_samples, axis=1)[:, ::-1])[:, :500]

In [ ]:
train_positives = np.hstack(
    [
        X[[col.user_code, col.movie_code]].astype(int).values,
        np.ones((X.shape[0], 1), dtype=int),
    ],
    dtype=int,
)

In [ ]:
def train_dataloader(X, negative_samples, n_negatives):
    row_negatives = X[col.user_code].values.repeat(n_negatives)
    col_negatives = np.random.randint(0, negative_samples.shape[1], row_negatives.shape[0])

    train_negatives = np.vstack(
        [
            row_negatives,
            negative_samples[row_negatives, col_negatives],
            np.zeros(row_negatives.shape[0])
        ]
    ).T

    train_data = torch.utils.data.DataLoader(
        np.vstack(
            [
                train_positives,
                train_negatives,
            ]
        ),
        batch_size=1024,
        shuffle=True,
        generator=torch.Generator(device=device),
    )
    
    return train_data

#### GMF

In [ ]:
model = GMFPointwise(*user_movie_matrix.shape, 8).to(device)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model
criterion = nn.BCEWithLogitsLoss()  # Choose your desired loss function
optimizer = optim.Adam(model.parameters(), lr=1e-3)
num_epochs = 20
n_negatives = 4

# Define the gradient clipping value
max_norm = 1.0

# Training loop
train_losses = []
for epoch in tqdm(range(num_epochs)):
    model.train()
    running_losses = 0

    train_data = train_dataloader(X, negative_samples, n_negatives)

    for inputs in tqdm(train_data):
        inputs = inputs.squeeze(1)
        uids, mids, labels = inputs.int().chunk(3, dim=1)
        optimizer.zero_grad()
        outputs = model(uids.squeeze(), mids.squeeze())
        loss = criterion(outputs, labels.float())
        loss.backward()

        # Perform gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        optimizer.step()

        train_losses.append(loss.detach().item())
        running_losses += loss.detach().item()

    epoch_loss = running_losses / len(train_data)

    retrieval = []

    model.eval()
    with torch.no_grad():
        for inputs in tqdm(test_data):
            uids, mids = inputs.chunk(2, dim=1)
            scores = model(uids.squeeze(), mids.squeeze())
            retrieval.append(scores.cpu().numpy())

    y_pred = np.take_along_axis(
        test_array,
        np.argsort(np.vstack(retrieval).reshape(-1, test_array.shape[1]))[:, ::-1],
        axis=1,
    )[:, :10]

    print(
        f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, NDCG: {ndcg_score(y_true, y_pred):.4f}"
    )